In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if physical_devices != []:
    print("Using GPU")
    for i in physical_devices:
        tf.config.experimental.set_memory_growth(i, True)
else:
    print("Using CPU")
    pass

# Model Architectures

## classical_cnn

In [ ]:
def classical_cnn(dim, output_neurons, output_activation):
    print("\nTRAINING ON classical_cnn MODEL:-")
    
    def block(tensor, conv_reps, n_filters):
        x = Conv2D(filters = n_filters, kernel_size = (3,3), padding = 'same')(tensor)
        x = LeakyReLU()(x)
        
        for i in range(conv_reps-1):
            x = Conv2D(filters = n_filters, kernel_size = (3,3), padding = 'same')(x)
            x = LeakyReLU()(x)
            
        x = MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'same')(x)
        
        return x

    input = Input(shape = dim)
        
    k = 8
    x = block(input, conv_reps = 1, n_filters = k)
    x = block(x, conv_reps = 1, n_filters = 2*k)
    x = block(x, conv_reps = 1, n_filters = 4*k)
    x = block(x, conv_reps = 1, n_filters = 8*k)
    x = block(x, conv_reps = 1, n_filters = 16*k)
    
    x = Flatten()(x)
    
    dense_reps = 2
    
    for i in range(dense_reps):
        x = Dense(128)(x)
        x = LeakyReLU()(x)
    
    output = Dense(output_neurons, output_activation)(x)  
    
    model = Model(inputs = input, outputs = output)
    
    return model

## mobile_inception

In [ ]:
def mobile_inception(dim, output_neurons, output_activation):
    print("\nTRAINING ON mobile_inception MODEL:-")

    
    def block(x, filters, reps):
        for _ in range(reps):
            # for low-level features
            t1 = Conv2D(filters[0], kernel_size = (1,1))(x)
            t1 = LeakyReLU()(t1)

            # for mid-level features
            t2 = DepthwiseConv2D(kernel_size = (3,3), strides = 1, padding = 'same')(x)
            t2 = LeakyReLU()(t2)
            t2 = Conv2D(filters[1], kernel_size = (1,1))(t2)
            t2 = LeakyReLU()(t2)

            # for high-level features
            t3 = DepthwiseConv2D(kernel_size = (5,5), strides = 1, padding = 'same')(x)
            t3 = LeakyReLU()(t3)
            t3 = Conv2D(filters[2], kernel_size = (1,1))(t3)
            t3 = LeakyReLU()(t3)

            # for most-significant features
            t4 = MaxPool2D(pool_size = (3,3), strides = 1, padding = 'same')(x)
            t4 = Conv2D(filters[3], kernel_size = (1,1))(t4)
            t4 = LeakyReLU()(t4)

            x = Concatenate()([t1, t2, t3, t4])
        
        return x
        
    input = Input(shape = dim)
    
    k = 16
    
    x = Conv2D(filters = k, kernel_size = (7,7), strides = 2, padding = 'same')(input)
    x = LeakyReLU()(x)
    x = MaxPool2D(pool_size = (3,3), strides = 2, padding = 'same')(x)
    
    x = DepthwiseConv2D(kernel_size = (3,3), strides = 1, padding = 'same')(x)
    x = LeakyReLU()(x)
    x = Conv2D(filters = 2*k, kernel_size = (1,1))(x)
    x = LeakyReLU()(x)
    x = MaxPool2D(pool_size = (2,2), strides = 2)(x)
    
    x = block(x, [k, k, k, k], reps = 2)
    x = MaxPool2D(pool_size = (2,2), strides = 2)(x)
    
    x = block(x, [2*k, 2*k, 2*k, 2*k], reps = 2)
    x = MaxPool2D(pool_size = (2,2), strides = 2)(x)

    x = block(x, [4*k, 4*k, 4*k, 4*k], reps = 2)
     
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    
    output = Dense(output_neurons, output_activation)(x)
    
    model = Model(inputs = input, outputs = output)    
    
    return model

# Choose Model Architecture

In [ ]:
h = int(input("Image Dimension(H or W): "))
w = h
dim = (h,w,3)

In [ ]:
output_neurons = int(input("Number of classes: "))

if output_neurons > 1:
    output_activation = 'softmax'
else:
    output_activation = 'sigmoid'

In [ ]:
print("Press 1 for classical_cnn")
print("Press 2 for mobile_inception")

model_select = int(input("\nChoose model: "))

if model_select == 1:
    model = classical_cnn(dim, output_neurons, output_activation)
if model_select == 2:
    model = mobile_inception(dim, output_neurons, output_activation)

# Summary

In [ ]:
model.summary()

In [ ]:
plot_model(
    model,
#     to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB", # TB -> Vertical; LR -> Horizontal
    expand_nested=False,
    dpi=300,
)